# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [67]:
# Libraries
import pandas as pd
import numpy as np

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [68]:
# your code here
Admission = pd.read_csv('Admission_Predict.csv')
Admission

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


Remove trailing spaces at the end of the column names if there are any.


In [69]:
# your code here
Admission.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR ', 'CGPA', 'Research', 'Chance of Admit '],
      dtype='object')

In [70]:
Admission.rename(columns = {'LOR ':'LOR', 'Chance of Admit ':'Chance of Admit'}, inplace = True)
Admission.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit'],
      dtype='object')

Let's evaluate the dataset by looking at the `head` function.

In [71]:
# your code here
Admission.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [72]:
# your code here

Admission.isnull().sum()


Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [73]:
# your code here
Admission = Admission.set_index('Serial No.')
Admission.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [74]:
# your code here
print(Admission['GRE Score'].dtype)
print(Admission['CGPA'].dtype)
Admission['Combine Score'] = Admission['GRE Score'].astype(str)+Admission['CGPA'].astype(str)
print(Admission['Combine Score'].nunique())
Admission.shape


int64
float64
385


(385, 9)

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

In [75]:
Admission.set_index(['GRE Score','CGPA'], inplace=True)


Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [76]:
# your code here
Admission.reset_index(inplace=True)
Admission.head()

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,Combine Score
0,337,9.65,118,4,4.5,4.5,1,0.92,3379.65
1,316,8.00,104,3,3.0,3.5,1,0.72,3168.0
2,322,8.67,110,3,3.5,2.5,1,0.80,3228.67
3,314,8.21,103,2,2.0,3.0,0,0.65,3148.21
4,330,9.34,115,5,4.5,3.0,1,0.90,3309.34


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [77]:
# your code here
Admission[(Admission['CGPA']>9)&(Admission['Research']==1)]

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,Combine Score
0,337,9.65,118,4,4.5,4.5,1,0.92,3379.65
4,330,9.34,115,5,4.5,3.0,1,0.90,3309.34
10,328,9.10,112,4,4.0,4.5,1,0.78,3289.1
19,328,9.50,116,5,5.0,5.0,1,0.94,3289.5
20,334,9.70,119,5,5.0,4.5,1,0.95,3349.7
...,...,...,...,...,...,...,...,...,...
379,329,9.23,111,4,4.5,4.0,1,0.89,3299.23
380,324,9.04,110,3,3.5,3.5,1,0.82,3249.04
381,325,9.11,107,3,3.0,3.5,1,0.84,3259.11
382,330,9.45,116,4,5.0,4.5,1,0.91,3309.45


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [78]:
# your code here
Admission[(Admission['CGPA']>9)&(Admission['SOP']<3.5)]['Chance of Admit'].mean()

0.8019999999999999

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [79]:
def standardize(col,col_mean,col_all_std):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    return (col- col_mean)/col_all_std
    # your code here

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [80]:
# your code here
Admission['CGPA_mean'] = np.mean(Admission['CGPA'])
Admission['LOR_mean'] = np.mean(Admission['LOR'])
Admission['GRE_mean'] = np.mean(Admission['GRE Score'])
Admission['CGPA_all_std'] = np.std(Admission['CGPA'])
Admission['LOR_all_std'] = np.std(Admission['LOR'])
Admission['GRE_all_std'] = np.std(Admission['GRE Score'])

Admission.head()

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,Combine Score,CGPA_mean,LOR_mean,GRE_mean,CGPA_all_std,LOR_all_std,GRE_all_std
0,337,9.65,118,4,4.5,4.5,1,0.92,3379.65,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093
1,316,8.00,104,3,3.0,3.5,1,0.72,3168.0,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093
2,322,8.67,110,3,3.5,2.5,1,0.80,3228.67,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093
3,314,8.21,103,2,2.0,3.0,0,0.65,3148.21,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093
4,330,9.34,115,5,4.5,3.0,1,0.90,3309.34,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093


In [81]:
Admission['CGPA_std'] =standardize(Admission['CGPA'],Admission['CGPA_mean'],Admission['CGPA_all_std'])
Admission['LOR_std'] =standardize(Admission['LOR'],Admission['LOR_mean'],Admission['LOR_all_std'])
Admission['GRE_std'] =standardize(Admission['GRE Score'],Admission['GRE_mean'],Admission['GRE_all_std'])

Admission.head()

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,Combine Score,CGPA_mean,LOR_mean,GRE_mean,CGPA_all_std,LOR_all_std,GRE_all_std,CGPA_std,LOR_std,GRE_std
0,337,9.65,118,4,4.5,4.5,1,0.92,3379.65,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093,1.750174,1.193197,1.755663
1,316,8.00,104,3,3.0,3.5,1,0.72,3168.0,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093,-0.992501,0.076840,-0.063450
2,322,8.67,110,3,3.5,2.5,1,0.80,3228.67,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093,0.121191,-1.039517,0.456297
3,314,8.21,103,2,2.0,3.0,0,0.65,3148.21,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093,-0.643433,-0.481338,-0.236698
4,330,9.34,115,5,4.5,3.0,1,0.90,3309.34,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093,1.234884,-0.481338,1.149292


We will generate the decision choice at random using the code below. Please run the cell.

In [82]:
# Libraries
from random import choices

In [83]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=Admission.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [84]:
# your code here
Admission['Deciding Column'] = decision_choice
Admission.head(10)

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,Combine Score,CGPA_mean,LOR_mean,GRE_mean,CGPA_all_std,LOR_all_std,GRE_all_std,CGPA_std,LOR_std,GRE_std,Deciding Column
0,337,9.65,118,4,4.5,4.5,1,0.92,3379.65,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093,1.750174,1.193197,1.755663,LOR_std
1,316,8.00,104,3,3.0,3.5,1,0.72,3168.0,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093,-0.992501,0.076840,-0.063450,CGPA_std
2,322,8.67,110,3,3.5,2.5,1,0.80,3228.67,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093,0.121191,-1.039517,0.456297,CGPA_std
3,314,8.21,103,2,2.0,3.0,0,0.65,3148.21,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093,-0.643433,-0.481338,-0.236698,LOR_std
4,330,9.34,115,5,4.5,3.0,1,0.90,3309.34,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093,1.234884,-0.481338,1.149292,GRE_std
5,321,8.20,109,3,3.0,4.0,1,0.75,3218.2,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093,-0.660055,0.635019,0.369672,CGPA_std
6,308,7.90,101,2,3.0,4.0,0,0.68,3087.9,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093,-1.158723,0.635019,-0.756445,GRE_std
7,302,8.00,102,1,2.0,1.5,0,0.50,3028.0,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093,-0.992501,-2.155874,-1.276191,LOR_std
8,323,8.60,108,3,3.5,3.0,0,0.45,3238.6,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093,0.004836,-0.481338,0.542921,GRE_std
9,327,9.00,111,4,4.0,4.5,1,0.84,3279.0,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093,0.669726,1.193197,0.889419,LOR_std


In [85]:
Admission['Deciding Number'] = Admission.lookup(Admission.index, Admission['Deciding Column'])
Admission.sample(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  """Entry point for launching an IPython kernel.


,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,Combine Score,CGPA_mean,LOR_mean,GRE_mean,CGPA_all_std,LOR_all_std,GRE_all_std,CGPA_std,LOR_std,GRE_std,Deciding Column,Deciding Number
235,311,8.48,104,2,2.5,3.5,0,0.71,3118.48,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093,-0.194632,0.076840,-0.496571,LOR_std,0.076840
339,297,7.67,98,2,2.5,3.0,0,0.59,2977.67,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093,-1.541036,-0.481338,-1.709313,CGPA_std,-1.541036
51,320,7.70,103,3,3.0,3.0,0,0.64,3207.7,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093,-1.491169,-0.481338,0.283048,LOR_std,-0.481338
70,314,8.90,106,3,3.0,5.0,0,0.74,3148.9,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093,0.503504,1.751376,-0.236698,CGPA_std,0.503504
160,313,8.27,102,3,2.0,3.0,0,0.64,3138.27,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093,-0.543699,-0.481338,-0.323323,CGPA_std,-0.543699
46,313,8.30,98,3,2.5,4.5,1,0.76,3138.3,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093,-0.493833,1.193197,-0.323323,GRE_std,-0.323323
380,324,9.04,110,3,3.5,3.5,1,0.82,3249.04,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093,0.736216,0.076840,0.629546,CGPA_std,0.736216
33,300,7.80,105,1,1.0,2.0,0,0.58,3007.8,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093,-1.324946,-1.597696,-1.449440,LOR_std,-1.597696
190,317,8.54,103,3,2.5,3.0,1,0.73,3178.54,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093,-0.094898,-0.481338,0.023175,LOR_std,-0.481338
111,299,8.62,102,3,4.0,3.5,0,0.56,2998.62,8.597091,3.431169,316.732468,0.601602,0.895771,11.544093,0.038080,0.076840,-1.536064,LOR_std,0.076840


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [86]:
# your code here
def decision(x):
  if x > 0.8:
    return 1
  else:
    return 0

Admission['Decision'] = Admission['Deciding Number'].apply(decision)
Admission.head()

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,Combine Score,CGPA_mean,...,GRE_mean,CGPA_all_std,LOR_all_std,GRE_all_std,CGPA_std,LOR_std,GRE_std,Deciding Column,Deciding Number,Decision
0,337,9.65,118,4,4.5,4.5,1,0.92,3379.65,8.597091,...,316.732468,0.601602,0.895771,11.544093,1.750174,1.193197,1.755663,LOR_std,1.193197,1
1,316,8.00,104,3,3.0,3.5,1,0.72,3168.0,8.597091,...,316.732468,0.601602,0.895771,11.544093,-0.992501,0.076840,-0.063450,CGPA_std,-0.992501,0
2,322,8.67,110,3,3.5,2.5,1,0.80,3228.67,8.597091,...,316.732468,0.601602,0.895771,11.544093,0.121191,-1.039517,0.456297,CGPA_std,0.121191,0
3,314,8.21,103,2,2.0,3.0,0,0.65,3148.21,8.597091,...,316.732468,0.601602,0.895771,11.544093,-0.643433,-0.481338,-0.236698,LOR_std,-0.481338,0
4,330,9.34,115,5,4.5,3.0,1,0.90,3309.34,8.597091,...,316.732468,0.601602,0.895771,11.544093,1.234884,-0.481338,1.149292,GRE_std,1.149292,1


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [87]:
# your code here
Admission['Decision'].value_counts()

0    302
1     83
Name: Decision, dtype: int64

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [91]:
# your code here
Admission.columns = Admission.columns.str.replace(" ", "_")
# There is not any non character symbols
Admission.columns = Admission.columns.str.lower()
Admission.columns

Index(['gre_score', 'cgpa', 'toefl_score', 'university_rating', 'sop', 'lor',
       'research', 'chance_of_admit', 'combine_score', 'cgpa_mean', 'lor_mean',
       'gre_mean', 'cgpa_all_std', 'lor_all_std', 'gre_all_std', 'cgpa_std',
       'lor_std', 'gre_std', 'deciding_column', 'deciding_number', 'decision'],
      dtype='object')

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [97]:
# your code here
def boost(row):
  if row['university_rating'] >= 4:
    return row['gre_score'] + 10
  else:
    return row['gre_score']

Admission['adjusted_gre'] = Admission.apply(boost,axis=1)
Admission.head()

,gre_score,cgpa,toefl_score,university_rating,sop,lor,research,chance_of_admit,combine_score,cgpa_mean,...,cgpa_all_std,lor_all_std,gre_all_std,cgpa_std,lor_std,gre_std,deciding_column,deciding_number,decision,adjusted_gre
0,337,9.65,118,4,4.5,4.5,1,0.92,3379.65,8.597091,...,0.601602,0.895771,11.544093,1.750174,1.193197,1.755663,LOR_std,1.193197,1,347
1,316,8.00,104,3,3.0,3.5,1,0.72,3168.0,8.597091,...,0.601602,0.895771,11.544093,-0.992501,0.076840,-0.063450,CGPA_std,-0.992501,0,316
2,322,8.67,110,3,3.5,2.5,1,0.80,3228.67,8.597091,...,0.601602,0.895771,11.544093,0.121191,-1.039517,0.456297,CGPA_std,0.121191,0,322
3,314,8.21,103,2,2.0,3.0,0,0.65,3148.21,8.597091,...,0.601602,0.895771,11.544093,-0.643433,-0.481338,-0.236698,LOR_std,-0.481338,0,314
4,330,9.34,115,5,4.5,3.0,1,0.90,3309.34,8.597091,...,0.601602,0.895771,11.544093,1.234884,-0.481338,1.149292,GRE_std,1.149292,1,340


In [100]:
bins = pd.cut(Admission['adjusted_gre'],4)
bins.value_counts()

(305.0, 320.0]     150
(320.0, 335.0]      90
(335.0, 350.0]      75
(289.94, 305.0]     70
Name: adjusted_gre, dtype: int64